In [2]:
import os
import sys
sys.path.append('/Users/aghavamp/Desktop/Projects')
sys.path.append('/Users/aghavamp/Desktop/Projects/bimanual_wrist')
sys.path.append('/Users/aghavamp/Desktop/Projects/Functional_Fusion')
import getpass
import importlib
from tqdm import tqdm

import scipy.io as sio
import rsatoolbox as rsa
from rsatoolbox.io import spm as spm_io
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import surfAnalysisPy as surf
import SUITPy as suit
import nibabel as nb
import nitools as nt
from matplotlib.cm import ScalarMappable
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from pathlib import Path
import seaborn as sns
import PcmPy as pcm
import Functional_Fusion.atlas_map as am
import Functional_Fusion.reliability as rel
import glob
import matplotlib.patches as patches

import utils
importlib.reload(utils)

# SET PATHS:
baseDir = os.path.join('/Users', getpass.getuser(), 'Desktop', 'Projects', 'bimanual_wrist', 'data', 'fMRI')
bidsDir = 'BIDS'
anatomicalDir = 'anatomicals'
freesurferDir = 'surfaceFreesurfer'
surfacewbDir = 'surfaceWB' 
behavDir = 'behavioural'
regDir = 'ROI'
atlasDir = '/Volumes/diedrichsen_data$/data/Atlas_templates/fs_LR_32'
analysisDir = os.path.join(os.path.dirname(os.path.dirname(baseDir)), 'analysis')


extract the region data and the descriptors for each subj and save it in analysis directory.

In [6]:
glm = 'single_jul30'
region_labels = [1, 2, 3, 4, 5, 6, 7, 8]
region_names = ['S1', 'M1', 'PMd', 'PMv', 'SMA', 'V1', 'SPLa', 'SPLp']
sn_list = [101,102,103,104,106,107,108,109,110,111,112,113,114,115]
hem = ['L','R']
atlas,_ = am.get_atlas('fs32k')

# Intrinsic ordering of the conditions:
extrinsic_names = ['lhand:0',  'lhand:60',  'lhand:120',  'lhand:180',  'lhand:240',  'lhand:300',
                   'rhand:0',  'rhand:60',  'rhand:120',  'rhand:180',  'rhand:240',  'rhand:300',
                   'bi:0_0',   'bi:0_60',   'bi:0_120',   'bi:0_180',   'bi:0_240',   'bi:0_300',
                   'bi:60_0',  'bi:60_60',  'bi:60_120',  'bi:60_180',  'bi:60_240',  'bi:60_300',
                   'bi:120_0', 'bi:120_60', 'bi:120_120', 'bi:120_180', 'bi:120_240', 'bi:120_300',
                   'bi:180_0', 'bi:180_60', 'bi:180_120', 'bi:180_180', 'bi:180_240', 'bi:180_300',
                   'bi:240_0', 'bi:240_60', 'bi:240_120', 'bi:240_180', 'bi:240_240', 'bi:240_300',
                   'bi:300_0', 'bi:300_60', 'bi:300_120', 'bi:300_180', 'bi:300_240', 'bi:300_300']

intrinsic_names = ['lhand:flx',     'lhand:flxup',    'lhand:extup',    'lhand:ext',    'lhand:extdn',    'lhand:flxdn',
                   'rhand:ext',     'rhand:extup',    'rhand:flxup',    'rhand:flx',    'rhand:flxdn',    'rhand:extdn',
                   'bi:flx_ext',    'bi:flx_extup',   'bi:flx_flxup',   'bi:flx_flx',   'bi:flx_flxdn',   'bi:flx_extdn',
                   'bi:flxup_ext',  'bi:flxup_extup', 'bi:flxup_flxup', 'bi:flxup_flx', 'bi:flxup_flxdn', 'bi:flxup_extdn',
                   'bi:extup_ext',  'bi:extup_extup', 'bi:extup_flxup', 'bi:extup_flx', 'bi:extup_flxdn', 'bi:extup_extdn',
                   'bi:ext_ext',    'bi:ext_extup',   'bi:ext_flxup',   'bi:ext_flx',   'bi:ext_flxdn',   'bi:ext_extdn',
                   'bi:extdn_ext',  'bi:extdn_extup', 'bi:extdn_flxup', 'bi:extdn_flx', 'bi:extdn_flxdn', 'bi:extdn_extdn',
                   'bi:flxdn_ext',  'bi:flxdn_extup', 'bi:flxdn_flxup', 'bi:flxdn_flx', 'bi:flxdn_flxdn', 'bi:flxdn_extdn']

cond_num = [0, 1, 2, 3, 4, 5,
            9, 8, 7, 6, 11, 10,
            15, 14, 13, 12, 17, 16,
            21, 20, 19, 18, 23, 22,
            27, 26, 25, 24, 29, 28,
            33, 32, 31, 30, 35, 34,
            39, 38, 37, 36, 41, 40,
            45, 44, 43, 42, 47, 46]

# Create mapping dictionary
extrinsic_to_intrinsic = dict(zip(extrinsic_names, intrinsic_names))
extrinsic_to_number = dict(zip(extrinsic_names, cond_num))

for k, r in enumerate(region_labels):
    for j, h in enumerate(hem):
        data = []
        cond_extrinsic_vec = []
        cond_intrinsic_vec = []
        cond_num_vec = []
        part_vec = []
        subject = []
        for i, sn in enumerate(sn_list):
            regressor_info = pd.read_table(os.path.join(baseDir, f'glm{glm}', f's{sn}', 'reginfo.tsv'))
            partitions = regressor_info['run'].values.flatten()
            conds = regressor_info['name'].values.flatten()
            conds = np.array([cond.strip() for cond in np.array(conds)])

            # ==========================================
            # DATA EXTRACTION
            # ==========================================
            #  Define atlas map
            white = os.path.join(baseDir, surfacewbDir, f's{sn}', f's{sn}.{h}.white.32k.surf.gii') # Individual white surface
            pial = os.path.join(baseDir, surfacewbDir, f's{sn}', f's{sn}.{h}.pial.32k.surf.gii') # Invividual pial surface
            mask = os.path.join(baseDir, f'glm{glm}', f's{sn}', 'mask.nii') # Mask in functional space for that subject

            # File names for data extraction
            nii_names = sorted(glob.glob(os.path.join(baseDir, f'glm{glm}', f's{sn}', "beta_*.nii")))
            
            beta = []
            # resMS = []
            # beta_white = []
            
            atlas_tmp = atlas.get_hemisphere(j)
            subatlas = atlas_tmp.get_subatlas_image(os.path.join(atlasDir,f'ROI.32k.{h}.label.gii'), value=r)
            amap = am.AtlasMapSurf(subatlas.vertex[0],white,pial,mask) # Atlas map
            # Compute the voxels in native space
            amap.build()
            
            # This extract all the relevant voxels in native space (use for RSA)
            beta = amap.extract_data_native(nii_names)
            idx_nan = np.isnan(beta).any(axis=0)
            beta = beta[:, ~idx_nan]
            
            # ========================================== 
            # store the data
            # ==========================================
            # select unimanual conditions:
            subject.append(sn)
            data.append(beta)
            cond_extrinsic_vec.append(conds)
            part_vec.append(partitions)
            
            # Convert conditions vector to intrinsic names
            cond_intrinsic_vec.append(np.array([extrinsic_to_intrinsic[cond] for cond in conds]))

            # Convert conditions vector to numbers
            cond_num_vec.append(np.array([extrinsic_to_number[cond] for cond in conds]))

            print(f'{region_names[k]}, hem{h}, s{sn}')

        # save the data of hem and region:
        # Convert to object array to handle different shapes
        data_obj = np.empty(len(data), dtype=object)
        for i in range(len(data)):
            data_obj[i] = data[i]

        np.savez(os.path.join(analysisDir, f'pwbeta_{region_names[k]}_{h}.npz'), 
                  data=data_obj, 
                  cond_extrinsic_vec=cond_extrinsic_vec, 
                  cond_intrinsic_vec=cond_intrinsic_vec,
                  cond_num_vec=cond_num_vec,
                  part_vec=part_vec,
                  subject=subject)



101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
101
102
103
104
106
107
108
109
110
111
112
113
114
115
